## Libraries 

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import pandas as pd
import numpy as np
print(tf.__version__)

2.4.1


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from keras import models
from keras import layers
from keras import optimizers
from PIL import Image
import sklearn
from sklearn.model_selection import train_test_split
from statistics import mean

## Data Loading

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd 'drive/MyDrive/Image Perception Group Project/Project/'

/content/drive/MyDrive/Image Perception Group Project/Project


In [ ]:
!dir

1.\ Get_data.ipynb		InceptionResnetv2.ipynb
2.\ Clean_data.ipynb		Multi_hot_encoded_data.csv
3.\ Organise_data.ipynb		Poster_New
4.\ Train-1.ipynb		Posters
5.\ Train-2.py			Posters2
6.\ Test-accuracy.ipynb		processed1.csv
7.\ Test_single_image.ipynb	processed2.csv
abhyuday1.h5			ResNet50_googlecolab_version.ipynb
Abhyuday-Get_data.ipynb		resnet50_model(2).h5
abhyuday.h5			Test_X.npy
CSV				Test_Y.npy
Dataset				Train_X.npy
DenseNet-169.ipynb		Train_Y.npy
densenet169wh_final.h5		Valid_X.npy
Emotions.ipynb			Valid_Y.npy
emotions_model.hdf5		VGG16.ipynb
Encoded_data_column_lookup.csv	X_test(1).npy
hsv-Copy1.ipynb			Y_test(1).npy
hsv.ipynb


In [ ]:
print("Processing train..")
X_train = np.load('Train_X.npy')
Y_train = np.load('Train_Y.npy')

Processing train..


In [ ]:
print("Processing valid..")
X_val = np.load('Valid_X.npy')
Y_val = np.load('Valid_Y.npy')

Processing valid..


In [ ]:
X_train.shape

(18284, 200, 150, 3)

In [ ]:
Y_val.shape

(3917, 10)

## Model

In [ ]:
num_classes = 10
from keras.applications.resnet50 import ResNet50

model = Sequential()
resnet50 = ResNet50(input_shape = (200,150,3), include_top = False)
model.add(resnet50)
model.add(MaxPooling2D())
model.add(Flatten())

# model.add(Dense(units=4096,activation="relu"))
# model.add(layers.Dropout(0.6))
model.add(Dense(units=1024,activation="relu"))
model.add(Dense(units=128,activation="relu"))
# model.add(layers.Dropout(0.5))
model.add(Dense(units=num_classes, activation="sigmoid"))
model.summary()



94773248/94765736 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 5, 2048)        23587712  
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 2, 2048)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12288)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              12583936  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 36,304,138
Trainable params: 36,251,018
Non

## Training

In [ ]:
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='resnet50_model.h5', mode='max', monitor= accuracy, verbose=2, save_best_only=True)
# earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', verbose=1, min_delta=1)
# callbacks_list = [checkpoint, earlystopping]
import keras.backend as K

THRESHOLD = 0.3
def accuracy(y_true, y_pred, threshold_shift=0.5-THRESHOLD):

    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred_bin, 0, 1)))


    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return accuracy
   
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='drive/MyDrive/Image Perception Group Project/Project/resnet50_model(2).h5', mode='max', monitor= 'val_accuracy', verbose=2, save_best_only=True)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', verbose=1, min_delta=1, patience = 5)    
callbacks_list = [checkpoint, earlystopping]

model.compile(optimizer=optimizers.Adam(lr=1e-4), loss='binary_crossentropy', metrics=[accuracy])

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,width_shift_range=0.2, height_shift_range=0.2, 
                         shear_range=0.15,horizontal_flip=True, fill_mode="nearest")

# train the network
EPOCHS=50
BS = 32

history = model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS),validation_data=(X_val, Y_val), 
                    steps_per_epoch=len(X_train) // BS, epochs=EPOCHS, callbacks = callbacks_list)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50


KeyboardInterrupt: ignored

In [ ]:
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'b', label='Training acc')
# plt.plot(epochs, val_acc, 'r', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()

# plt.figure()

# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

# plt.show()

In [5]:
review_model = tf.keras.models.load_model('resnet50_model(2).h5')

In [7]:
review_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 5, 2048)        23587712  
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 2, 2048)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12288)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              12583936  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 36,304,138
Trainable params: 36,251,018
Non-trainable params: 53,120
_______________________________________

In [ ]:
!dir

1.\ Get_data.ipynb		InceptionResnetv2.ipynb
2.\ Clean_data.ipynb		Multi_hot_encoded_data.csv
3.\ Organise_data.ipynb		Poster_New
4.\ Train-1.ipynb		Posters
5.\ Train-2.py			Posters2
6.\ Test-accuracy.ipynb		ResNet50_googlecolab_version.ipynb
7.\ Test_single_image.ipynb	resnet50_model(2).h5
abhyuday1.h5			Test_X.npy
Abhyuday-Get_data.ipynb		Test_Y.npy
abhyuday.h5			Train_X.npy
CSV				Train_Y.npy
Dataset				Valid_X.npy
DenseNet-169.ipynb		Valid_Y.npy
densenet169wh_final.h5		VGG16.ipynb
Encoded_data_column_lookup.csv


In [ ]:
def arrange_data(df):
    
    image_data = []
    img_paths = np.asarray(df.iloc[:, 0]) #First column is the image paths
    
    for i in tqdm(range(len(img_paths))):
              
        img = image.load_img(img_paths[i],target_size=(200,150,3))
        img = image.img_to_array(img)
        img = img/255
        image_data.append(img)
        
    
    X = np.array(image_data)
    Y = np.array(df.iloc[:,1:11])
    
    print("Shape of images:", X.shape)
    print("Shape of labels:", Y.shape)
    
    return X, Y

In [ ]:
print("Processing test..")
test_df = pd.read_csv("./Dataset/Test.csv", delimiter=" ")
X_test, Y_test = arrange_data (test_df)


  0%|          | 0/3917 [00:00<?, ?it/s]

Processing test..



100%|██████████| 3917/3917 [26:09<00:00,  2.50it/s]


Shape of images: (3917, 200, 150, 3)
Shape of labels: (3917, 10)


In [9]:
print("Processing test..")
X_test = np.load('Test_X.npy')
# Y_test = np.load('Test_Y.npy')
Y_test = np.load('Test_Y.npy', allow_pickle = True)

Processing test..


In [15]:
X_test

array([[[[0.77254903, 0.79607844, 0.78039217],
         [0.7764706 , 0.8       , 0.78431374],
         [0.78431374, 0.80784315, 0.7921569 ],
         ...,
         [0.10196079, 0.10588235, 0.08627451],
         [0.10196079, 0.10588235, 0.08627451],
         [0.10196079, 0.10588235, 0.08627451]],

        [[0.78039217, 0.8039216 , 0.7882353 ],
         [0.78039217, 0.8039216 , 0.7882353 ],
         [0.7882353 , 0.8117647 , 0.79607844],
         ...,
         [0.09803922, 0.10196079, 0.08235294],
         [0.09803922, 0.10196079, 0.08235294],
         [0.09803922, 0.10196079, 0.08235294]],

        [[0.78431374, 0.80784315, 0.7921569 ],
         [0.7882353 , 0.8117647 , 0.79607844],
         [0.79607844, 0.81960785, 0.8039216 ],
         ...,
         [0.09803922, 0.10196079, 0.08235294],
         [0.09803922, 0.10196079, 0.08235294],
         [0.09803922, 0.10196079, 0.08235294]],

        ...,

        [[0.91764706, 0.93333334, 0.92941177],
         [0.91764706, 0.93333334, 0.92941177]

In [14]:
Y_test

array([[0, 0, 0, ..., 0, 1, 'Crime|Drama|Thriller'],
       [0, 0, 0, ..., 0, 1, 'Horror|Thriller'],
       [0, 0, 0, ..., 0, 0, 'Documentary'],
       ...,
       [1, 0, 0, ..., 0, 0, 'Action|Comedy|Crime'],
       [0, 0, 0, ..., 0, 0, 'Comedy|Crime|Sport'],
       [0, 0, 0, ..., 0, 1, 'Horror|Thriller']], dtype=object)

In [11]:
X_test = np.asarray(X_test.astype('float32'))

In [13]:
test_loss, test_acc = review_model.evaluate(X_test, Y_test) 

ValueError: ignored

In [ ]:
np.save('X_test(1).npy', X_test) 

In [ ]:
np.save('Y_test(1).npy', Y_test) 